In [1]:
import psycopg2
import numpy as np
import pandas as pd
import datetime as dt
import cx_Oracle
import csv

In [8]:
con =  cx_Oracle.connect('comprasweb_ij/Xxxx1234!@10.10.2.223/MKTG')
print(con.version)

df = pd.read_sql_query('''
SELECT	
			COD_EMPRESA EM_ID_CLIENTE,
			sum(s.importe) AS importe_total
		FROM	
			DWHPRO.FACT_ENTERPRISE_SALES s
		WHERE	
			to_date(s.FECHA_IMPUTACION, 'yyyymmdd') BETWEEN CURRENT_DATE -366 AND CURRENT_DATE -1
			having sum(s.importe) > 0
		GROUP BY
			s.COD_EMPRESA 
''', con)
importe_df = df
con.close()

11.2.0.3.0


In [77]:
importe_df.head()

,EM_ID_CLIENTE,IMPORTE_TOTAL
0,1734087401618,99.00
1,1726336430170,4320.10
2,8600738420,10224.37
3,1731177223596,409.00
4,1731703728654,316.00


In [4]:
con =  cx_Oracle.connect('comprasweb_ij/Xxxx1234!@10.10.2.223/MKTG')
print(con.version)

df = pd.read_sql_query('''
SELECT 
EM_ID_CLIENTE,
EM_ID,
CASE WHEN EM_PACK = 1 THEN 'WEB' ELSE 'COMERCIAL PS' END AS TYPE_EM,
EM_NOMBRE,
EM_EMAIL,
eg_id,
eg.EG_NOMBRE,
a.eu_email,
ep.EP_NOMBRE_PUBLICO,
ep.ep_id
FROM 
	EMPLEO_ES.EMPRESAS_MATRICES em 
INNER JOIN	
	EMPLEO_ES.PD_PACKS ppp ON ppp.ID = em.EM_PACK
INNER JOIN 
	EMPLEO_ES.EMPRESAS_GRUPOS eg ON eg.EG_MATRIZ = em.EM_ID
INNER JOIN 
	--EMPLEO_ES.EMPRESAS_PERFILES ep ON ep.EP_ID = eg.EG_PERFIL
	EMPLEO_ES.EMPRESAS_PERFILES ep ON ep.ep_matriz = em_id
INNER JOIN 
(
SELECT * FROM 
(SELECT
RANK() over(PARTITION BY EU_GRUPO ORDER BY EU_LAST_LOGIN DESC) AS ranking 
,eu.EU_EMAIL
,eu.EU_GRUPO
FROM 
EMPLEO_ES.EMPRESAS_USUARIOS eu 
WHERE EU_ESTADO = 1
)t 
WHERE t.ranking = 1
)a 
ON a.EU_GRUPO = eg_id
	WHERE 
em.EM_ESTADO = 1 AND em.EM_PRUEBA <> 1 
''', con)
empresas_df = df
con.close()

11.2.0.3.0


In [76]:
empresas_df.head()

,EM_ID_CLIENTE,EM_ID,TYPE_EM,EM_NOMBRE,EM_EMAIL,EG_ID,EG_NOMBRE,EU_EMAIL,EP_NOMBRE_PUBLICO,EP_ID
0,8600610350,48386492,WEB,Lf Channel,rrhh@lfchannel.com,48386494,Grupo Administrador,ncazorla@lfchannel.com,Lf Channel,48386493
1,8600610350,48386492,WEB,Lf Channel,rrhh@lfchannel.com,48386494,Grupo Administrador,ncazorla@lfchannel.com,LF Channel-,48386495
2,8600610352,48386518,COMERCIAL PS,World Software Services,asoto@wssgroup.com,74302616,ADMINISTRADOR,recmad3@wssgroup.com,World Software Services,48386519
3,8600610352,48386518,COMERCIAL PS,World Software Services,asoto@wssgroup.com,74302616,ADMINISTRADOR,recmad3@wssgroup.com,Empresa consultora internacional de servicios ...,1018617766
4,8600610352,48386518,COMERCIAL PS,World Software Services,asoto@wssgroup.com,74302616,ADMINISTRADOR,recmad3@wssgroup.com,World Software Services (Barcelona),48386521


In [99]:
empresas_df.shape

(855988, 10)

In [81]:
con =  cx_Oracle.connect('comprasweb_ij/Xxxx1234!@10.10.2.223/MKTG')
print(con.version)

df = pd.read_sql_query('''
--SELECT 
	--t.ID_EXPERIENCIA_NORM,
	--t.DESC_EXPERIENCIA_NORMALIZADO,
--	t.ID_OFERTA ep_id,
--	ROUND(sum(t.RATINGEVENTOBJECTSCORE)/
--	count(t.EXPERIENCE_ID),1) promedio,
--	count(t.EXPERIENCE_ID) cantidad
--FROM 
--(--t --SI NO FUNCIONA AGREGAR DESDE ACA A JUPYTER Y SACAR PROMEDIOS DESDE EL NOTEBOOK
SELECT *
FROM 
(
SELECT 
	RATINGEVENTOBJECTTARGETID EXPERIENCE_ID,
	s.DT,
	TRUNC(s.RATINGEVENTOBJECTINSTANT) AS fecha_review,
	s.RATINGEVENTOBJECTSCORE,
	s.REVIEWNAME,
	s.REVIEWVALUE
FROM DWHPRO.COMPANY_REVIEW s 
)t 
pivot (
min(REVIEWVALUE) value FOR REVIEWNAME IN ('title', 'description')
)
INNER JOIN 
DWHPRO.EXPERIENCIAS_COMPANY_NORM e ON ID_EXPERIENCIA = EXPERIENCE_ID
INNER JOIN dwhpro.OFERTAS_COMPANY_NORM o ON o.DESC_OFERTA_NORMALIZADO = e.DESC_EXPERIENCIA_NORMALIZADO
WHERE 1=1
AND o.NORMALIZATION_MODEL IS NOT NULL AND e.NORMALIZATION_MODEL IS NOT NULL 
AND o.SCORE >= 1 AND e.SCORE >= 1
AND ID_EXPERIENCIA_NORM <> -1 AND o.ID_PUESTO_NORM <> -1 
--)t 
--GROUP BY 
--t.ID_EXPERIENCIA_NORM,
--t.DESC_EXPERIENCIA_NORMALIZADO,
--t.ID_OFERTA
''', con)
rating_df = df
con.close()

11.2.0.3.0


In [83]:
rating_df.shape

(449778, 44)

In [23]:
#Load the data 
df_csv = pd.read_csv('id_aprobados_v1.csv')#, encoding='UTF-8', sep=',', header=None)

In [71]:
id_aprobados = df_csv.drop(df_csv.columns[0],axis=1)

In [74]:
id_aprobados.head()

,EXPERIENCE_ID
0,8548593044
1,34101032986
2,34180496500
3,34180730112
4,34170367384


In [84]:
rating_df = rating_df.merge(id_aprobados,how='inner',on = 'EXPERIENCE_ID') 

In [85]:
rating_df.shape

(369421, 44)

In [86]:
rating_df.head()

,EXPERIENCE_ID,DT,FECHA_REVIEW,RATINGEVENTOBJECTSCORE,'title'_VALUE,'description'_VALUE,ID,TYPE,ACTION,ID_EXPERIENCIA_NORM,...,SCORE,TENANT,NORM_TYPE,PROVIDER_ID,PROVIDER_CHANNEL,PUBLISHED,SCHEMA,DT,FECHA_CARGA,NORMALIZATION_MODEL
0,207416242,2019-04-01,2019-04-01,3,No Aplica,No Aplica,db1afb3d-2e17-42fe-b8eb-e747f7ab47c9,NormalizationEvent,Create,6393,...,1,INFOJOBS,COMPANY_PROFILE,infojobs,other,2019-10-09 15:34:44,http://schema.spain.schibsted.io/events/infojo...,2019-10-09,2019-11-03 22:36:18,2.0
1,222691743,2019-02-24,2019-02-24,4,Positiva,positiva,717a43f5-804c-42b8-8f6b-acbeadb062c5,NormalizationEvent,Create,4,...,1,INFOJOBS,COMPANY_PROFILE,infojobs,other,2019-10-09 16:19:55,http://schema.spain.schibsted.io/events/infojo...,2019-10-09,2019-11-03 22:36:18,2.0
2,225785693,2019-09-08,2019-09-08,4,Buenas primas.,"Nunca se sale a la hora, tienes que limpiar la...",96268aba-7c6c-4af2-a428-ae7f21c1d4db,NormalizationEvent,Create,1521,...,1,INFOJOBS,COMPANY_PROFILE,infojobs,other,2019-10-09 16:51:38,http://schema.spain.schibsted.io/events/infojo...,2019-10-09,2019-11-03 22:36:18,2.0
3,225785693,2019-09-08,2019-09-08,4,Buenas primas.,"Nunca se sale a la hora, tienes que limpiar la...",96268aba-7c6c-4af2-a428-ae7f21c1d4db,NormalizationEvent,Create,1521,...,1,INFOJOBS,COMPANY_PROFILE,infojobs,other,2019-10-09 17:10:02,http://schema.spain.schibsted.io/events/infojo...,2019-10-09,2019-11-03 22:36:18,2.0
4,231636891,2019-08-01,2019-08-01,3,Correcto,Correcto,ab241ac9-6fd3-49d9-bbfc-d0d3ca98f7c6,NormalizationEvent,Create,1292,...,1,INFOJOBS,COMPANY_PROFILE,infojobs,other,2019-10-09 15:58:29,http://schema.spain.schibsted.io/events/infojo...,2019-10-09,2019-11-03 22:36:18,2.0


In [93]:
grouped = rating_df.groupby(['ID_OFERTA'])

metrics = grouped.agg({'EXPERIENCE_ID': pd.Series.nunique,
                       'RATINGEVENTOBJECTSCORE': pd.Series.mean
                      })

In [94]:
metrics.reset_index(inplace=True)

In [95]:
metrics.rename(columns={'ID_OFERTA': 'EP_ID', 'RATINGEVENTOBJECTSCORE': 'RATING'}, inplace=True)

In [96]:
metrics.head()

,EP_ID,EXPERIENCE_ID,RATING
0,48386493,1,4.00
1,48386495,1,4.00
2,48386519,6,4.50
3,48386547,1,5.00
4,48386571,316,3.85


In [97]:
perfiles_score = empresas_df.merge(metrics,how='inner',on = 'EP_ID') 

In [98]:
perfiles_score.head()

,EM_ID_CLIENTE,EM_ID,TYPE_EM,EM_NOMBRE,EM_EMAIL,EG_ID,EG_NOMBRE,EU_EMAIL,EP_NOMBRE_PUBLICO,EP_ID,EXPERIENCE_ID,RATING
0,8600610350,48386492,WEB,Lf Channel,rrhh@lfchannel.com,48386494,Grupo Administrador,ncazorla@lfchannel.com,Lf Channel,48386493,1,4.0
1,8600610350,48386492,WEB,Lf Channel,rrhh@lfchannel.com,48386494,Grupo Administrador,ncazorla@lfchannel.com,LF Channel-,48386495,1,4.0
2,8600610352,48386518,COMERCIAL PS,World Software Services,asoto@wssgroup.com,74302616,ADMINISTRADOR,recmad3@wssgroup.com,World Software Services,48386519,6,4.5
3,8600610354,48386544,COMERCIAL PS,GRUPO SOLUTIO,rrhh@solutioconsultores.com,2246987818,Perfil oculto,ldiaz@gruposolutio.com,"Solutio Consultores, S.A.",48386547,1,5.0
4,8600610354,48386544,COMERCIAL PS,GRUPO SOLUTIO,rrhh@solutioconsultores.com,180486280,SOLUTIO,ilopez@gruposolutio.com,"Solutio Consultores, S.A.",48386547,1,5.0


In [100]:
perfiles_score.shape

(13276, 12)

In [101]:
perfiles_score_importe = perfiles_score.merge(importe_df,how='inner',on = 'EM_ID_CLIENTE')

In [102]:
perfiles_score_importe.shape

(10726, 13)

In [103]:
perfiles_score_importe.head()

,EM_ID_CLIENTE,EM_ID,TYPE_EM,EM_NOMBRE,EM_EMAIL,EG_ID,EG_NOMBRE,EU_EMAIL,EP_NOMBRE_PUBLICO,EP_ID,EXPERIENCE_ID,RATING,IMPORTE_TOTAL
0,8600610352,48386518,COMERCIAL PS,World Software Services,asoto@wssgroup.com,74302616,ADMINISTRADOR,recmad3@wssgroup.com,World Software Services,48386519,6,4.5,19518.00
1,8600610354,48386544,COMERCIAL PS,GRUPO SOLUTIO,rrhh@solutioconsultores.com,2246987818,Perfil oculto,ldiaz@gruposolutio.com,"Solutio Consultores, S.A.",48386547,1,5.0,34488.46
2,8600610354,48386544,COMERCIAL PS,GRUPO SOLUTIO,rrhh@solutioconsultores.com,180486280,SOLUTIO,ilopez@gruposolutio.com,"Solutio Consultores, S.A.",48386547,1,5.0,34488.46
3,8600610354,48386544,COMERCIAL PS,GRUPO SOLUTIO,rrhh@solutioconsultores.com,2246987818,Perfil oculto,ldiaz@gruposolutio.com,SOLUTIO,180481587,4,2.5,34488.46
4,8600610354,48386544,COMERCIAL PS,GRUPO SOLUTIO,rrhh@solutioconsultores.com,180486280,SOLUTIO,ilopez@gruposolutio.com,SOLUTIO,180481587,4,2.5,34488.46


In [104]:
perfiles_score_importe.to_csv('perfiles_score_importe.csv', index= False)
#, sep=';')